In [1]:
#Loading Libraries
import numpy as np
import pandas as pd

In [2]:
# Read Data

data=pd.read_csv('axcs_train.csv')

data.head()

,ID,URL,Date,Title,InfoTheory,CompVis,Math,Abstract
0,cs-9301111,arxiv.org/abs/cs/9301111,1989-12-31,Nested satisfiability,0,0,0,Nested satisfiability A special case of the s...
1,cs-9301112,arxiv.org/abs/cs/9301112,1990-03-31,A note on digitized angles,0,0,0,A note on digitized angles We study the confi...
2,cs-9301113,arxiv.org/abs/cs/9301113,1991-07-31,Textbook examples of recursion,0,0,0,Textbook examples of recursion We discuss pro...
3,cs-9301114,arxiv.org/abs/cs/9301114,1991-10-31,Theory and practice,0,0,0,Theory and practice The author argues to Sili...
4,cs-9301115,arxiv.org/abs/cs/9301115,1991-11-30,Context-free multilanguages,0,0,0,Context-free multilanguages This article is a...


In [3]:
data.loc[:,'label'] = pd.Series(data[['InfoTheory','CompVis','Math']].values.tolist(), index=data.index)

In [4]:
data.tail()

,ID,URL,Date,Title,InfoTheory,CompVis,Math,Abstract,label
54726,no-14128412,arxiv.org/abs/1412.8412,2014-12-31,Sanitization of Call Detail Records via Differ...,0,0,0,Sanitization of Call Detail Records via Diffe...,"[0, 0, 0]"
54727,no-14053347,arxiv.org/abs/1405.3347,2014-12-31,Self-dual codes and quadratic residue codes ov...,1,0,1,Self-dual codes and quadratic residue codes o...,"[1, 0, 1]"
54728,no-150100212,arxiv.org/abs/1501.00212,2014-12-31,Set-merging for the Matching Algorithm of Mica...,0,0,0,Set-merging for the Matching Algorithm of Mic...,"[0, 0, 0]"
54729,no-150100203,arxiv.org/abs/1501.00203,2014-12-31,Small Cell Traffic Balancing Over Licensed and...,0,0,0,Small Cell Traffic Balancing Over Licensed an...,"[0, 0, 0]"
54730,no-14101009,arxiv.org/abs/1410.1009,2014-12-31,Uplink Scheduling for LTE Video Surveillance S...,0,0,0,Uplink Scheduling for LTE Video Surveillance ...,"[0, 0, 0]"


In [5]:
data = data[['Abstract','label']]

In [6]:
data.head()

,Abstract,label
0,Nested satisfiability A special case of the s...,"[0, 0, 0]"
1,A note on digitized angles We study the confi...,"[0, 0, 0]"
2,Textbook examples of recursion We discuss pro...,"[0, 0, 0]"
3,Theory and practice The author argues to Sili...,"[0, 0, 0]"
4,Context-free multilanguages This article is a...,"[0, 0, 0]"


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import MultiLabelBinarizer

In [8]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(data.label)
Y = multilabel_binarizer.transform(data.label)

In [9]:
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(data.Abstract)

In [10]:
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)

In [12]:
from sklearn.model_selection import train_test_split
x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, Y, test_size=0.2, random_state=9000)

In [13]:
#OneVsRest Model
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import hamming_loss

In [14]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float(len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

def print_score(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print("Hamming loss: {}".format(hamming_loss(y_pred, y_test_tfidf)))
    print("Hamming score: {}".format(hamming_score(y_pred, y_test_tfidf)))

In [16]:
nb_clf = MultinomialNB()
sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=6, tol=None)
lr = LogisticRegression()

In [20]:
clf = OneVsRestClassifier(nb_clf)
clf.fit(x_train_tfidf, y_train_tfidf)
y_pred = clf.predict(x_test_tfidf)
print_score(y_pred, classifier)

Clf:  LogisticRegression
Hamming loss: 0.0797935507444962
Hamming score: 0.9202064492555038
---


C:\Users\swayam.mittal\AppData\Local\Continuum\anaconda3\envs\tfdeeplearning\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label 0 is present in all training examples.
  str(classes[c]))


In [19]:
clf = OneVsRestClassifier(sgd)
clf.fit(x_train_tfidf, y_train_tfidf)
y_pred = clf.predict(x_test_tfidf)
print_score(y_pred, classifier)

Clf:  LogisticRegression
Hamming loss: 0.12167717182789806
Hamming score: 0.8783228281721019
---


C:\Users\swayam.mittal\AppData\Local\Continuum\anaconda3\envs\tfdeeplearning\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label 0 is present in all training examples.
  str(classes[c]))


In [18]:
clf = OneVsRestClassifier(lr)
clf.fit(x_train_tfidf, y_train_tfidf)
y_pred = clf.predict(x_test_tfidf)
print_score(y_pred, classifier)

C:\Users\swayam.mittal\AppData\Local\Continuum\anaconda3\envs\tfdeeplearning\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label 0 is present in all training examples.
  str(classes[c]))


Clf:  LogisticRegression
Hamming loss: 0.0720745409701288
Hamming score: 0.9279254590298712
---


C:\Users\swayam.mittal\AppData\Local\Continuum\anaconda3\envs\tfdeeplearning\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
